In [1]:
pip install termcolor

In [2]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [13]:
import time
import random
from termcolor import colored
import emoji
from IPython.display import display, Image as IPImage
import urllib.request

# UserId=Moinul
# Password=Moinul909
# Login Credentials
USER_ID = "Moinul"
PASSWORD = "Moinul909"

# Function for user authentication
def authenticate():
    print(colored("Please log in to access the system", 'cyan'))
    entered_user = input("Enter UserId: ")
    entered_password = input("Enter Password: ")

    if entered_user == USER_ID and entered_password == PASSWORD:
        print(colored("Login successful! Welcome to the Vehicle Rental System.", 'green'))
        return True
    else:
        print(colored("Invalid UserId or Password. Access denied.", 'red'))
        return False

# Database to store vehicles
vehicle_db = {}

# Base Class: Vehicle
class Vehicle:
    def __init__(self, make, model, rental_price, image_url, code=None):
        self.make = make
        self.model = model
        self.rental_price = rental_price
        self.is_available = True  # Track availability
        self.image_url = image_url  # Store image URL for the vehicle
        self.code = code if code else self.generate_code()  # Assign unique code

    @staticmethod
    def generate_code():
        """Generate a unique code for each vehicle"""
        return f"V-{random.randint(1000, 9999)}"

    def display_image(self):
        """Display the image of the vehicle from online URL"""
        try:
            with urllib.request.urlopen(self.image_url) as url:
                display(IPImage(url.read()))
        except Exception as e:
            print(colored(f"Error loading image: {e}", 'red'))

    def rent(self, duration):
        """Calculates the total rental cost"""
        if self.is_available:
            total_cost = duration * self.rental_price
            self.is_available = False
            vehicle_db[self.code].is_available = False  # Update in database
            return total_cost
        else:
            return None

    def return_vehicle(self):
        """Mark vehicle as available again and update database"""
        self.is_available = True
        vehicle_db[self.code].is_available = True  # Update in database
        print(colored(f"{self.model} (Code: {self.code}) has been returned and is now available.", 'green'))

    def __str__(self):
        status = "Available" if self.is_available else "Not Available"
        return f"Code: {self.code} | {self.make} {self.model} | Price per day: ${self.rental_price} | {status}"

# Car Subclass
class Car(Vehicle):
    def __init__(self, make, model, rental_price, seats, image_url, code=None):
        super().__init__(make, model, rental_price, image_url, code)
        self.seats = seats

    def __str__(self):
        return f"Car: {self.make} {self.model} | Seats: {self.seats} | {super().__str__()}"

# Bike Subclass
class Bike(Vehicle):
    def __init__(self, make, model, rental_price, bike_type, image_url, code=None):
        super().__init__(make, model, rental_price, image_url, code)
        self.bike_type = bike_type

    def __str__(self):
        return f"Bike: {self.make} {self.model} | Type: {self.bike_type} | {super().__str__()}"

# Truck Subclass
class Truck(Vehicle):
    def __init__(self, make, model, rental_price, load_capacity, image_url, code=None):
        super().__init__(make, model, rental_price, image_url, code)
        self.load_capacity = load_capacity

    def __str__(self):
        return f"Truck: {self.make} {self.model} | Load Capacity: {self.load_capacity} tons | {super().__str__()}"

# Rental Management System
class RentalSystem:
    def __init__(self):
        self.vehicles = {}

    def add_vehicle(self, vehicle):
        """Add a new vehicle to the system with a unique code"""
        self.vehicles[vehicle.code] = vehicle
        vehicle_db[vehicle.code] = vehicle
        print(colored(f"Vehicle {vehicle.make} {vehicle.model} added with Code: {vehicle.code}.", 'green'))

    def display_available_vehicles(self):
        """Display all available vehicles"""
        print(colored("\nAvailable Vehicles for Rent:", 'cyan'))
        for vehicle in self.vehicles.values():
            if vehicle.is_available:
                print(colored(vehicle, 'yellow'))

    def rent_vehicle(self):
        """Handle vehicle rental process"""
        self.display_available_vehicles()
        selected_code = input("\nEnter the Code of the vehicle you want to rent: ")
        duration = int(input("Enter rental duration (in days): "))
        
        vehicle = self.vehicles.get(selected_code)
        if vehicle:
            vehicle.display_image()  # Show vehicle image
            total_cost = vehicle.rent(duration)
            if total_cost:
                print(colored(f"\n{vehicle.model} rented for {duration} days. Total cost: ${total_cost} {emoji.emojize(':moneybag:')} {emoji.emojize(':credit_card:')}", 'green'))
                return
            else:
                print(colored(f"\n{vehicle.model} is currently not available.", 'red'))
        else:
            print(colored("\nVehicle with this code not found. Please try again.", 'red'))

    def return_vehicle(self):
        """Handle vehicle return process"""
        selected_code = input("\nEnter the Code of the vehicle you want to return: ")
        
        vehicle = self.vehicles.get(selected_code)
        if vehicle:
            vehicle.return_vehicle()
        else:
            print(colored("\nVehicle with this code not found. Please try again.", 'red'))

    def enroll_vehicle(self):
        """Enroll a new vehicle into the drive-sharing system"""
        print(colored("\nEnroll a New Vehicle in Drive Sharing:", 'cyan'))
        make = input("Enter Vehicle Make: ")
        model = input("Enter Vehicle Model: ")
        price = float(input("Enter Rental Price per Day: "))
        vehicle_type = input("Enter Vehicle Type (Car, Bike, Truck): ")
        image_url = input("Enter Image URL for the Vehicle: ")

        if vehicle_type.lower() == "car":
            seats = int(input("Enter number of seats: "))
            new_vehicle = Car(make, model, price, seats, image_url)
        elif vehicle_type.lower() == "bike":
            bike_type = input("Enter Bike Type (Sport, Cruiser, etc.): ")
            new_vehicle = Bike(make, model, price, bike_type, image_url)
        elif vehicle_type.lower() == "truck":
            load_capacity = int(input("Enter load capacity (tons): "))
            new_vehicle = Truck(make, model, price, load_capacity, image_url)
        else:
            print(colored("Invalid vehicle type. Try again.", 'red'))
            return

        self.add_vehicle(new_vehicle)

# Main Application
if __name__ == "__main__":
    if authenticate():
        rental_system = RentalSystem()

        # Sample vehicles for rent with online images
        rental_system.add_vehicle(Car("Toyota", "Corolla", 40, 5, "https://example.com/car_image.png"))
        rental_system.add_vehicle(Bike("Yamaha", "R15", 20, "Sport", "https://example.com/bike_image.png"))
        rental_system.add_vehicle(Truck("Ford", "F150", 100, 2, "https://example.com/truck_image.png"))

        # Main system loop
        while True:
            print(colored("\n--- Vehicle Rental and Drive Sharing System ---", 'blue'))
            print("1. View Available Vehicles")
            print("2. Rent a Vehicle")
            print("3. Return a Vehicle")
            print("4. Enroll a New Vehicle (Drive Sharing)")
            print("5. Exit")

            choice = input("\nEnter your choice: ")

            if choice == "1":
                rental_system.display_available_vehicles()
            elif choice == "2":
                rental_system.rent_vehicle()
            elif choice == "3":
                rental_system.return_vehicle()
            elif choice == "4":
                rental_system.enroll_vehicle()
            elif choice == "5":
                print(colored("Thank you for using the Vehicle Rental and Drive Sharing System! Goodbye!", 'green'))
                break
            else:
                print(colored("Invalid choice. Please try again.", 'red'))

            time.sleep(1)  # Pause for a better user experience


Please log in to access the system


Enter UserId:  Moinul
Enter Password:  Moinul909


Login successful! Welcome to the Vehicle Rental System.
Vehicle Toyota Corolla added with Code: V-9653.
Vehicle Yamaha R15 added with Code: V-9856.
Vehicle Ford F150 added with Code: V-2424.

--- Vehicle Rental and Drive Sharing System ---
1. View Available Vehicles
2. Rent a Vehicle
3. Return a Vehicle
4. Enroll a New Vehicle (Drive Sharing)
5. Exit



Enter your choice:  1



Available Vehicles for Rent:
Car: Toyota Corolla | Seats: 5 | Code: V-9653 | Toyota Corolla | Price per day: $40 | Available
Bike: Yamaha R15 | Type: Sport | Code: V-9856 | Yamaha R15 | Price per day: $20 | Available
Truck: Ford F150 | Load Capacity: 2 tons | Code: V-2424 | Ford F150 | Price per day: $100 | Available

--- Vehicle Rental and Drive Sharing System ---
1. View Available Vehicles
2. Rent a Vehicle
3. Return a Vehicle
4. Enroll a New Vehicle (Drive Sharing)
5. Exit


...


Available Vehicles for Rent:
Car: Toyota Corolla | Seats: 5 | Code: V-9653 | Toyota Corolla | Price per day: $40 | Available
Bike: Yamaha R15 | Type: Sport | Code: V-9856 | Yamaha R15 | Price per day: $20 | Available
Truck: Ford F150 | Load Capacity: 2 tons | Code: V-2424 | Ford F150 | Price per day: $100 | Available



Enter the Code of the vehicle you want to rent:  V-9653
Enter rental duration (in days):  47


Error loading image: HTTP Error 500: Internal Server Error

Corolla rented for 47 days. Total cost: $1880 :moneybag: 💳

--- Vehicle Rental and Drive Sharing System ---
1. View Available Vehicles
2. Rent a Vehicle
3. Return a Vehicle
4. Enroll a New Vehicle (Drive Sharing)
5. Exit



Enter your choice:  5


Thank you for using the Vehicle Rental and Drive Sharing System! Goodbye!
